# Main Notebook

This Jupyter notebook contains the basic code needed for the functionality for obtaining numerical results for one dimensional potentials, as well as an example of how you would get started obtaining results using the 1D harmonic oscillator.

## Discretized Schrödinger Function

In [1]:
# Necessary libraries
import numpy as np
import scipy as sp

In [12]:
def discretized_schrodinger(potentialFunction, initialBoundaryPoint: float, finalBoundaryPoint: float, numPosSteps: int, numEigVals: int, mass: float, hbar=1.0):
    # Calculate discretized points and spacing for [a,a+dx,a+2dx,...,b-dx,b]
    xs = np.linspace(initialBoundaryPoint,finalBoundaryPoint,numPosSteps+2)
    dx = np.diff(xs)[0]

    # Compute potential along discretized points
    Vx = potentialFunction(xs)

    # Calculate main diagonal, trimming out end points where we should have psi(a)=psi(b)=0
    main_diag = (np.power(dx,-2.0) + mass * Vx)[1:-1]
    # Calculate sub/supradiagonal terms, constants defined by dx
    off_diag  = np.power(dx,-2.0) * -0.5 * np.ones(len(main_diag)-1)

    # Solve eigensystem using assumption matrix is symmetric tridiagonal,
    # only return first numEigvals to cut down on unnecessary computation
    eigvals, eigstates = sp.linalg.eigh_tridiagonal(
        d            = main_diag,
        e            = off_diag,
        eigvals_only = False,
        select       = 'i',
        select_range = (0,numEigVals-1)
    )

    # Returns two elements including list of energy eigenvalues with shape (numEigVals,)
    # as well as a list of eigenstates with shape (numEigVals,numPosSteps)
    return (hbar/mass)*eigvals, eigstates.T

## Harmonic Oscillator Example

Here we look at obtaining the first ten energy eigenvalues for the potential
$$V(x)=\frac{1}{2}m\omega^2 x^2$$
which should have exact eigenvalues given by
$$E_n=\hbar \omega \left(n+\frac{1}{2}\right)$$

In [18]:
# Set up parameters

# Reduced Planck's Constant, scaled to order unity here
hbar  = 6.626
# Angular frequency defining potential
w     = 3.5
# Left boundary point, should be negative
a     = -15
# Right boundary point, should be positive
b     = 15
# Number of mesh points between boundaries, must be int
N     = int(1e5)
# Number of eigenstates/eigenvalues to return
n     = 5
# Mass of particle considered
m     = 4

# Define the potential function for harmonic oscillator
def harmonic_oscillator(x, omega, mass): return (mass/2)*np.power(omega*x,2)

# Establish true energy eigenvalues for comparison
true_HO_vals = [hbar*w*(n+0.5) for n in range(0,n)]

# Obtain results from main function
HO_vals, HO_states = discretized_schrodinger(lambda x: harmonic_oscillator(x,w,m),
                                            a,
                                            b,
                                            N,
                                            n,
                                            m,
                                            hbar)

for val, true_val in zip(HO_vals, true_HO_vals):
    print(f'Numerical Result: {val:.4f}')
    print(f'True exact Result: {true_val:.4f}\n')

Numerical Result: 11.5955
True exact Result: 11.5955

Numerical Result: 34.7865
True exact Result: 34.7865

Numerical Result: 57.9775
True exact Result: 57.9775

Numerical Result: 81.1685
True exact Result: 81.1685

Numerical Result: 104.3595
True exact Result: 104.3595

